In [37]:
import pandas as pd
import requests
from openpyxl import load_workbook

In [38]:


urls = {
    "https://prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS%20DE%20ITBI%20PAGAS%20%282023%29%20ago.xlsx":"GUIAS_ITBI_2023.xlsx",
    "https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/fazenda/arquivos/XLSX/GUIAS_DE_ITBI_PAGAS_12-2022.xlsx":"GUIAS_ITBI_2022.xlsx"
}

for url , name in urls.items():
    
    response = requests.get(url)

    if response.status_code == 200:
        file_name = name

        with open(file_name, 'wb') as file:
            file.write(response.content)
        
        print(f"Arquivo {file_name} baixado com sucesso.")

        workbook = load_workbook(file_name)
        sheet = workbook.active

    else:
        print(f"Falha ao baixar o arquivo {url}.")


Arquivo GUIAS_ITBI_2023.xlsx baixado com sucesso.
Arquivo GUIAS_ITBI_2022.xlsx baixado com sucesso.


In [48]:

arquivos = ['GUIAS_ITBI_2022',
            'GUIAS_ITBI_2023']

dados_acumulados = pd.DataFrame()

for arquivo in arquivos:
    arquivo = pd.ExcelFile('/Users/joaosequeira/sp_apartments_for_sale/sp_apartments_for_sale/'+ arquivo +'.xlsx')

    for aba_nome in arquivo.sheet_names:
        df_aba = arquivo.parse(aba_nome)
        
        dados_acumulados = pd.concat([dados_acumulados, df_aba], ignore_index=True)





In [49]:

col_mapping = {
    "N° do Cadastro (SQL)": "Registration Number",
    "Nome do Logradouro": "Street Name",
    "Número": "Number",
    "Complemento": "Complement",
    "Bairro": "Neighborhood",
    "Referência": "Reference",
    "CEP": "Postal Code",
    "Natureza de Transação": "Transaction Nature",
    "Valor de Transação (declarado pelo contribuinte)": "Transaction Value",
    "Data de Transação": "Transaction Date",
    "Valor Venal de Referência": "Reference Property Value",
    "Proporção Transmitida (%)": "Transmitted Proportion",
    "Valor Venal de Referência (proporcional)": "Proportional Reference Property Value",
    "Base de Cálculo adotada": "Adopted Calculation Base",
    "Tipo de Financiamento": "Financing Type",
    "Valor Financiado": "Financed Value",
    "Cartório de Registro": "Registration Office",
    "Matrícula do Imóvel": "Property Deed",
    "Situação do SQL": "SQL Status",
    "Área do Terreno (m2)": "Land Area",
    "Testada (m)": "Frontage (m)",
    "Fração Ideal": "Ideal Fraction",
    "Área Construída (m2)": "Built Area",
    "Uso (IPTU)": "Use",
    "Descrição do uso (IPTU)": "Use Description",
    "Padrão (IPTU)": "Standard",
    "ACC (IPTU)": "ACC",
    "ACC (IPTU).1": "ACC_1",
    "CAMPO": "Field",
    "PREENCHIMENTO NA DTI (Delcaração de Transações Imobiliárias)": "Filling in DTI",
    "CONCEITO": "Concept",
    "TIPO": "Type",
    "TAMANHO": "Size",
    "DOMÍNIO (Valores válidos)": "Domain",
    "CONDIÇÕES": "Conditions",
    "OBSERVAÇÕES": "Observations",
    "USO": "Use",
    "DESCRIÇÃO": "Description",
    "PADRÃO": "Standard"
}


col_mapping = {k: v.lower().replace(' ', '_') for k, v in col_mapping.items()}

dados_acumulados.rename(columns=col_mapping, inplace=True)

dados_acumulados.columns = dados_acumulados.columns.str.replace(r"[^\w\s]", "").str.replace(" ", "_")



In [66]:

treated_db = dados_acumulados[['registration_number','number','complement','neighborhood',
                                    'postal_code', 'transaction_nature', 'transaction_value','transaction_date','built_area','use_description','transmitted_proportion']]

treated_db.isnull().sum()

registration_number         209
number                      209
complement                36437
neighborhood              96369
postal_code                 210
transaction_nature          209
transaction_value           209
transaction_date            209
built_area                  209
use_description             209
transmitted_proportion      209
dtype: int64

In [67]:
treated_db.dropna(how='all', inplace=True)

treated_db.isnull().sum()

/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_82536/1143776486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treated_db.dropna(how='all', inplace=True)


registration_number           0
number                        0
complement                36228
neighborhood              96160
postal_code                   1
transaction_nature            0
transaction_value             0
transaction_date              0
built_area                    0
use_description               0
transmitted_proportion        0
dtype: int64

In [69]:
final_db = treated_db[(treated_db['transaction_nature'] == '1.Compra e venda') & (treated_db['transmitted_proportion'] == 100.00)]

final_db

,registration_number,number,complement,neighborhood,postal_code,transaction_nature,transaction_value,transaction_date,built_area,use_description,transmitted_proportion
0,1.712150e+10,695.0,AP 94 E 2 VGS,VL ANDRADE,5727230.0,1.Compra e venda,720000.0,2021-12-21 00:00:00,178.0,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),100.0
11,5.101800e+09,503.0,CASA B,VILA BELA,3144080.0,1.Compra e venda,500000.0,2021-12-21 00:00:00,65.0,RESIDÊNCIA,100.0
32,6.111300e+09,921.0,NaN,NaN,3639000.0,1.Compra e venda,410000.0,2021-12-21 00:00:00,135.0,RESIDÊNCIA,100.0
34,6.902901e+09,17.0,NaN,V PAULICEIA,2302030.0,1.Compra e venda,650000.0,2021-12-27 00:00:00,119.0,RESIDÊNCIA,100.0
37,1.831200e+10,100.0,CASA 16 E VG,RESID VILLA FIORE,5782470.0,1.Compra e venda,413000.0,2021-12-22 00:00:00,63.0,RESIDÊNCIA,100.0
...,...,...,...,...,...,...,...,...,...,...,...
260782,1.103380e+10,251.0,CASA 5,NaN,3671000.0,1.Compra e venda,345000.0,2023-07-28 00:00:00,60.0,RESIDÊNCIA,100.0
260783,2.000302e+09,665.0,AP 84,TORRE A,1137010.0,1.Compra e venda,1450000.0,2023-07-31 00:00:00,239.0,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),100.0
260784,1.960371e+10,300.0,AP 32 TORRE B,BELENZINHO,3015010.0,1.Compra e venda,400000.0,2023-06-30 00:00:00,88.0,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),100.0
260788,4.720802e+09,113.0,AP 12 E 2 VGS,JABAQUARA,4306090.0,1.Compra e venda,784000.0,2023-06-29 00:00:00,154.0,APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL),100.0


In [76]:

translation_mapping = {
    'APARTAMENTO EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)': 'Apartment in Condominium (Requires Ideal Fraction)',
    'RESIDÊNCIA': 'Residence',
    'GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE USO EXCLUSIVAMENTE RESIDENCIAL (EXIGE FRAÇÃO IDEAL)': 'Garage (Autonomous Unit) in Residential Condominium (Requires Ideal Fraction)',
    'ESCRITÓRIO/CONSULTÓRIO EM CONDOMÍNIO (UNID AUTÔNOMA EXIGE FRAÇÃO IDEAL)': 'Office/Clinic in Condominium (Autonomous Unit Requires Ideal Fraction)',
    'FLAT DE USO COMERCIAL (SEMELHANTE À HOTEL)': 'Commercial Use Flat (Similar to Hotel)',
    'LOJA': 'Store',
    'FLAT RESIDENCIAL EM CONDOMÍNIO (EXIGE FRAÇÃO IDEAL)': 'Residential Flat in Condominium (Requires Ideal Fraction)',
    'TERRENO': 'Land',
    'GARAGEM (EXCLUSIVE EM PRÉDIO EM CONDOMÍNIO)': 'Garage (Exclusive in Condominium Building)',
    'RESIDÊNCIA COLETIVA, EXCLUSIVE CORTIÇO (MAIS DE UMA RESIDÊNCIA NO LOTE)': 'Collective Residence, Exclusive Slum (More Than One Residence on the Lot)',
    'OUTRAS EDIFICAÇÕES DO TIPO, COM UTILIZAÇÃO MÚLTIPLA': 'Other Buildings of the Type, with Multiple Use',
    'LOJA E RESIDÊNCIA (PREDOMINÂNCIA COMERCIAL)': 'Store and Residence (Commercial Predominance)',
    'RESIDÊNCIA E OUTRO USO (PREDOMINÂNCIA RESIDENCIAL)': 'Residence and Other Use (Residential Predominance)',
    'PRÉDIO DE ESCRITÓRIO OU CONSULTÓRIO, NÃO EM CONDOMÍNIO, COM OU SEM LOJAS': 'Office or Clinic Building, Not in Condominium, with or Without Stores',
    'LOJA EM EDIFÍCIO EM CONDOMÍNIO (UNIDADE AUTÔNOMA EXIGE FRAÇÃO IDEAL)': 'Store in Condominium Building (Autonomous Unit Requires Ideal Fraction)',
    'GARAGEM (UNIDADE AUTÔNOMA) EM EDIFÍCIO EM CONDOMÍNIO DE ESCRITÓRIOS, CONSULTÓRIOS OU MISTO (EXIGE FRAÇÃO IDEAL)': 'Garage (Autonomous Unit) in Office, Clinic, or Mixed-Use Condominium (Requires Ideal Fraction)',
    'ESCOLA': 'School',
    'POSTO DE SERVIÇO': 'Service Station',
    'OFICINA': 'Workshop',
    'INDÚSTRIA': 'Industry',
    'PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO MISTO (APARTAMENTOS E ESCRITÓRIOS E/OU CONSULTÓRIOS), COM OU SEM LOJA (PREDOMINÂNCIA RESIDENCIAL)': 'Apartment Building, Not in Condominium, Mixed Use (Apartments and Offices/Clinics), with or Without Store (Residential Predominance)',
    'HOTEL, PENSÃO OU HOSPEDARIA': 'Hotel, Boarding House or Hostel',
    'GARAGEM (UNIDADE AUTÔNOMA) DE PRÉDIO DE GARAGENS (EXIGE FRAÇÃO IDEAL)': 'Garage (Autonomous Unit) in Garage Building (Requires Ideal Fraction)',
    'TEMPLO': 'Temple',
    'ARMAZÉNS GERAIS E DEPÓSITOS': 'General Warehouses and Depots',
    'CORTIÇO (HABITAÇÃO COLETIVA SUBNORMAL)': 'Slum (Subnormal Collective Housing)',
    'PRÉDIO DE APARTAMENTO, NÃO EM CONDOMÍNIO, DE USO EXCLUSIVAMENTE RESIDENCIAL': 'Apartment Building, Not in Condominium, for Exclusive Residential Use',
    'OUTRAS EDIFICAÇÕES DO TIPO COM UTILIZAÇÃO MÚLTIPLA': 'Other Buildings of the Type with Multiple Use',
    'PRÉDIO COM USO MISTO, PREDOMINÂNCIA DE USO NÃO RESIDENCIAL, NÃO EM CONDOMÍNIO': 'Mixed-Use Building, Predominantly Non-Residential Use, Not in Condominium',
    'ASILO, ORFANATO, CRECHE, SEMINÁRIO OU CONVENTO': 'Nursing Home, Orphanage, Daycare, Seminary or Convent',
    'CINEMA, TEATRO, CASA DE DIVERSÃO, CLUBE OU CONGÊNERE': 'Cinema, Theater, Entertainment Venue, Club or Similar',
    'HOSPITAL, AMBULATÓRIO, CASA DE SAÚDE E ASSEMELHADOS': 'Hospital, Outpatient Clinic, Health Clinic and Similar',
    'ESTAÇÃO RADIOEMISSORA, DE TELEVISÃO OU EMPRESA JORNALÍSTICA': 'Radio Broadcasting Station, Television Station or News Agency',
    '1.Compra e venda':'buy_and_sell',
    ' ': 'Unknown'
}

final_db['use_description'] = final_db['use_description'].replace(translation_mapping)
final_db['transaction_nature'] = final_db['transaction_nature'].replace(translation_mapping)


/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_82536/4154326083.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_db['use_description'] = final_db['use_description'].replace(translation_mapping)
/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_82536/4154326083.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_db['transaction_nature'] = final_db['transaction_nature'].replace(translation_mapping)


In [78]:
final_db.to_csv('apartments_for_sale_sao_paulo.csv', index=False)